<a href="https://colab.research.google.com/github/zpsy-hub/IoT-Data-Generation-for-Smart-Logistic/blob/main/MO_IT148_Homework_IoT_Data_Simulation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Logistic Tracking Data Generator

This script generates a synthetic dataset for smart logistic tracking applications.
It creates a dataset with information about packages, their locations, status,
and environmental conditions during transit.

</br>

**Data Description:**

The dataset includes the following columns:

* **timestamp:** The date and time of the record.
* **package_id:** A unique identifier for each package.
* **location:** The geographical location of the package (latitude, longitude).
* **status:** The current status of the package (e.g., In Transit, Delivered, Pending, Delayed).
* **temperature:** The temperature surrounding the package during transit (in Celsius).
* **humidity:** The humidity level surrounding the package during transit (in percentage).

</br>

**Use Cases:**

This dataset can be used for various purposes, including:

* **Real-time Package Tracking:** Monitor the location and status of packages in real-time.
* **Delivery Optimization:** Analyze delivery routes and times to identify areas for improvement.
* **Condition Monitoring:** Track the temperature and humidity experienced by packages during transit to ensure product quality.
* **Predictive Analytics:** Build models to predict potential delays or delivery issues based on historical data.
* **Supply Chain Management:** Gain insights into the movement of goods and optimize inventory levels.


---



In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

num_records = 100  # Adjust this number as needed

# Example for Smart Logistic Tracking
data = []

for _ in range(num_records):
    record = {
        "timestamp": datetime.now() - timedelta(minutes=np.random.randint(0, 1440)),  # Random timestamp in the last 24 hours
        "package_id": f"PKG{np.random.randint(10000, 99999)}",  # Random package ID
        "location": f"{np.random.randint(-90, 90)}, {np.random.randint(-180, 180)}",  # Random latitude, longitude
        "status": np.random.choice(["In Transit", "Delivered", "Pending", "Delayed"]),  # Random status
        "temperature": round(np.random.uniform(-20, 30), 1),  # Random temperature in Celsius
        "humidity": round(np.random.uniform(0, 100), 1)  # Random humidity percentage
    }
    data.append(record)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save dataset
df.to_csv("logistics_data.csv", index=False)
df.to_json("logistics_data.json", orient="records")

# Display first few rows
df.head()

,timestamp,package_id,location,status,temperature,humidity
0,2025-05-06 09:14:20.043505,PKG91505,"14, 161",Pending,21.0,57.3
1,2025-05-05 18:16:20.043797,PKG22254,"-50, -52",Delayed,27.7,89.2
2,2025-05-05 18:44:20.044005,PKG81911,"-46, 85",In Transit,22.0,8.5
3,2025-05-05 22:54:20.044084,PKG77011,"60, -141",Delayed,22.4,37.9
4,2025-05-05 17:41:20.044153,PKG39716,"-80, 155",In Transit,-8.5,1.6


In [ ]:
from google.colab import drive
   drive.mount('/content/drive')